<a href="https://colab.research.google.com/github/moj-a/VQE_Task/blob/master/VQE_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from numpy import kron
from random import random
from scipy.optimize import minimize

from qiskit import *
from qiskit.circuit.library.standard_gates import U2Gate
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.algorithms import NumPyEigensolver

In [3]:
# Decomposing the given hamiltonian matrix:

def decomp(H):
    """
    decomposing the matrix to sum of Pauli terms
    :param H: 4*4 matrix.
    :params p_x, p_y, p_z ,p_I: are pauli matrices.
    :symbols: list of the pauli matrices that we use to represent the different terms of the decomposition. 
    :param term: different part of the decomposition.
    :param coef: coeficent for each term in the decomposition.
    """
   
    p_x = np.array([[0, 1],  [ 1, 0]])
    p_y = np.array([[0, -1j],[1j, 0]])
    p_z = np.array([[1, 0],  [0, -1]])
    p_I = np.array([[1, 0],  [ 0, 1]])
    P = [p_I, p_x, p_y, p_z]
    symbols = ['I', 's_x', 's_y', 's_z']
    for i in range(4):
        for j in range(4):
            term = symbols[i] + ' \otimes ' + symbols[j]
            coef_ij = 0.25 * np.dot(kron(P[i], P[j]).conjugate().transpose(), H).trace()
            if coef_ij != 0.0:
                print ( coef_ij.real, term)


In [4]:
# The given hamiltonian matrix:


H = np.matrix([[1,0,0,0], 
                      [0, 0, -1, 0], 
                      [0,-1,0,0], 
                      [0,0,0,1]])

In [5]:
H_decomp = decomp(H)

[[0.5]] I \otimes I
[[-0.5]] s_x \otimes s_x
[[-0.5]] s_y \otimes s_y
[[0.5]] s_z \otimes s_z


In [6]:
# Using Weighted Pauli Operator to reperesnt the hamiltonian 

pauli_dict = {
    'paulis': [{"coeff": {"imag": 0.0, "real": 0.5}, "label": "II"},
              {"coeff": {"imag": 0.0, "real": 0.5}, "label": "ZZ"},
              {"coeff": {"imag": 0.0, "real": -0.5}, "label": "XX"},
              {"coeff": {"imag": 0.0, "real": -0.5}, "label": "YY"}
              ]
}
H2=WeightedPauliOperator.from_dict(pauli_dict)

In [7]:
# Using Qiskit's NumPyEigensolver class to find the smallest eigenvalue of the given Hamiltonian with a classical algorithm. 
# Only as a reference for comparing with the result of my own VQE 

exact_result = NumPyEigensolver(H2).run()
print("exact_result:",exact_result)
reference_energy = min(np.real(exact_result.eigenvalues))
print('The exact ground state energy is: {}'.format(reference_energy))

exact_result: {'eigenvalues': array([-1.-2.76098236e-17j]), 'eigenstates': ListOp([VectorStateFn(Statevector([0.        +4.44522891e-18j, 0.58961433-3.90326707e-01j,
             0.58961433-3.90326707e-01j, 0.        +5.55111512e-17j],
            dims=(2, 2)), coeff=1.0, is_measurement=False)], coeff=1.0, abelian=False)}
The exact ground state energy is: -1.0000000000000002


In [14]:
#I only considerd RX as my variational parameter.

def quantum_state_preparation(circuit, parameters):
    q = circuit.qregs[0] # q is the quantum register where the info about qubits is stored
    circuit.h(0)
    circuit.cx(0, 1)
    circuit.rx(parameters[0], 0)
    return circuit

In [15]:
def vqe_circuit(parameters, measure):
    """
    Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'ZZ' stands for Z \otimes Z measurement.
    :return: quantum circuit.
    """
    circuit = QuantumCircuit(2)

    # quantum state preparation
    circuit = quantum_state_preparation(circuit, parameters)

    # measurement
    if measure == 'ZZ':
        circuit.measure_all()

    elif measure == 'XX':
        circuit.barrier()
        circuit.u2(0, np.pi, 0)
        circuit.u2(0, np.pi, 1)
        circuit.measure_all()

    elif measure == 'YY':
        circuit.barrier()
        circuit.u2(0, np.pi/2, 0)
        circuit.u2(0, np.pi/2, 1)
        circuit.measure_all()

    else:
        raise ValueError('Not valid input for measurement: input should be "XX" or "YY" or "ZZ"')

    return circuit

In [16]:
# Finding the expectation values of a Pauli operators: 

def quantum_module(parameters, measure):
    # measure
    if measure == 'II':
        return 1
    elif measure == 'ZZ':
        circuit = vqe_circuit(parameters, 'ZZ')
    elif measure == 'XX':
        circuit = vqe_circuit(parameters, 'XX')
    elif measure == 'YY':
        circuit = vqe_circuit(parameters, 'YY')
    else:
        raise ValueError('Not valid input for measurement: input should be "II" or "XX" or "ZZ" or "YY"')
    
    shots = 8192
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()

    
    # expectation value estimation from counts
    expectation_value = 0
    for measure_result in counts:
        if measure_result == '00':
          sign = +1
        elif measure_result == '11':
          sign = +1
        elif measure_result == '01':
          sign = -1
        elif measure_result == '10':
          sign = -1
        expectation_value += sign * counts[measure_result] / shots
        
    return expectation_value

In [17]:
# Creating a dictionary from the WeightedPauliOperator object:
 
def pauli_operator_to_dict(pauli_operator):
    """
    from WeightedPauliOperator return a dict:
    {'II': 0.5, 'XX': -0.5, 'YY': -0.5, 'ZZ': 0.5}.
    :param pauli_operator: qiskit's WeightedPauliOperator
    :return: a dict in the desired form.
    """
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}

    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff

    return paulis_dict
pauli_dict = pauli_operator_to_dict(H2)

In [18]:
pauli_dict

{'II': 0.5, 'XX': -0.5, 'YY': -0.5, 'ZZ': 0.5}

In [19]:
# Multiplying the expectation values of Pauli operators by their corresponding coefficients, and then add them all together:

def vqe(parameters):
        
    # quantum_modules
    quantum_module_II = pauli_dict['II'] * quantum_module(parameters, 'II')
    quantum_module_ZZ = pauli_dict['ZZ'] * quantum_module(parameters, 'ZZ')
    quantum_module_XX = pauli_dict['XX'] * quantum_module(parameters, 'XX')
    quantum_module_YY = pauli_dict['YY'] * quantum_module(parameters, 'YY')
    
    # summing the measurement results
    classical_adder = quantum_module_II + quantum_module_ZZ + quantum_module_XX + quantum_module_YY
    
    return classical_adder

In [20]:
#Comparing the exact ground state and the estimated ground state from my VQE:

parameters_array = np.array([np.pi])
tol = 1e-3 # tolerance for optimization precision.


vqe_result = minimize(vqe, parameters_array, method="Powell", tol=tol)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))

The exact ground state energy is: -1.0000000000000002
The estimated ground state energy from VQE algorithm is: -1.0
